In [1]:
%pip install pandas 


^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
""" import os
import requests
from dotenv import load_dotenv

# Load .env file
load_dotenv()
API_KEY = os.getenv("PRINTIFY_API_KEY")

BASE_URL = "https://api.printify.com/v1/"
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(BASE_URL + "shops.json", headers=headers)

if response.status_code != 200:
    print(f"❌ Error: {response.status_code}")
    print(response.text)
else:
    print("✅ Success! Your shop details:")
    print(response.json())
 """

' import os\nimport requests\nfrom dotenv import load_dotenv\n\n# Load .env file\nload_dotenv()\nAPI_KEY = os.getenv("PRINTIFY_API_KEY")\n\nBASE_URL = "https://api.printify.com/v1/"\nheaders = {"Authorization": f"Bearer {API_KEY}"}\n\nresponse = requests.get(BASE_URL + "shops.json", headers=headers)\n\nif response.status_code != 200:\n    print(f"❌ Error: {response.status_code}")\n    print(response.text)\nelse:\n    print("✅ Success! Your shop details:")\n    print(response.json())\n '

In [1]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("PRINTIFY_API_KEY")
SHOP_ID = "23711392"  # your actual Shop ID
BASE_URL = "https://api.printify.com/v1/"
headers = {"Authorization": f"Bearer {API_KEY}"}


In [2]:

def get_all_products(shop_id):
    url = f"{BASE_URL}shops/{shop_id}/products.json"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Error {resp.status_code}: {resp.text}")
    return resp.json().get("data", [])

In [3]:
def summarize_variants(variants):
    colors, sizes = set(), set()
    for v in variants:
        for attr in v.get("attributes", []):
            if attr.get("name") == "color":
                colors.add(attr.get("value"))
            elif attr.get("name") == "size":
                sizes.add(attr.get("value"))
    return f"{len(colors)} colors • {len(sizes)} sizes • Total {len(variants)} variants"

In [4]:
def transform_products(products):
    backend_data, frontend_data = [], []

    for p in products:
        # Extract first image and first variant price
        first_image = p.get("images", [{}])[0].get("src", "")
        variants = p.get("variants", [])
        base_price = variants[0].get("price", 0) / 100 if variants else 0
        
        
          # --- BACKEND FULL STRUCTURE ---
        backend_data.append({
            "product_id": p.get("id"),
            "title": p.get("title"),
            "description": p.get("description"),
            "blueprint_id": p.get("blueprint_id"),
            "print_provider_id": p.get("print_provider_id"),
            "visible": p.get("visible"),
            "variants": variants,
            "images": p.get("images", []),
            "tags": p.get("tags", []),
            "base_price": base_price
        })

        # --- FRONTEND SUBSET ---
        frontend_data.append({
            "product_id": p.get("id"),
            "title": p.get("title"),
            "image_url": first_image,
            "price": f"${base_price:.2f}",
            "variant_summary": summarize_variants(variants),
            "status": "Published" if p.get("visible") else "Unpublished",
            "inventory": "All in stock" if p.get("visible") else "Unavailable"
        })

    return backend_data, frontend_data

In [5]:

# Run locally
products = get_all_products(SHOP_ID)
backend_data, frontend_data = transform_products(products)

# Save JSONs
with open("backend_full_data.json", "w") as f:
    json.dump(backend_data, f, indent=4)

with open("frontend_ready_data.json", "w") as f:
    json.dump(frontend_data, f, indent=4)

print("✅ Saved backend_full_data.json (complete API data)")
print("✅ Saved frontend_ready_data.json (UI-clean data)")

✅ Saved backend_full_data.json (complete API data)
✅ Saved frontend_ready_data.json (UI-clean data)
